<a href="https://colab.research.google.com/github/dylanwalker/MGSC496/blob/main/MGSC496_QuoteScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here is the full code to create and run the Scrapy quotes scraper.
Use ***Runtime->Run All***

In [ ]:
%%capture
!pip install scrapy

In [ ]:
!scrapy startproject quotescraper

In [ ]:
%%writefile /content/quotescraper/quotescraper/spiders/quotespider.py
import scrapy
from quotescraper.items import QuotescraperItem

class QuotespiderSpider(scrapy.Spider):
    name = 'quotespider'
    allowed_domains = ['quotes.toscrape.com']
    start_urls = ['http://quotes.toscrape.com/']

    def parse(self, response):
        for quote_sel in response.xpath('//div[@class="quote"]'):
          quote = quote_sel.xpath('span[@class="text"]/text()').get()
          author = quote_sel.xpath('span/small/text()').get()
          tags = quote_sel.xpath('div[@class="tags"]/a[@class="tag"]/text()').getall()
          yield QuotescraperItem(quote=quote, author=author,tags=tags)
        rel_next_page_link = response.xpath('//li[@class="next"]/a/@href').get()
        if rel_next_page_link is not None:
          next_page_link = response.urljoin(rel_next_page_link)
          yield scrapy.Request(url=next_page_link, callback=self.parse)

Overwriting /content/quotescraper/quotescraper/spiders/quotespider.py


In [ ]:
%%writefile /content/quotescraper/quotescraper/items.py
# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy


class QuotescraperItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    quote = scrapy.Field()
    author = scrapy.Field()
    tags = scrapy.Field()
    


Overwriting /content/quotescraper/quotescraper/items.py


In [ ]:
!cd /content/quotescraper && scrapy crawl quotespider -O quotes.json